In [16]:
import gymnasium as gym
from pogema import GridConfig
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=30
)

env = gym.make("Pogema-v0",grid_config=grid_config)

a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


#### Optuna Integration

In [17]:
""" Optuna example that optimizes the hyperparameters of
a reinforcement learning agent using A2C implementation from Stable-Baselines3
on a Gymnasium environment.

This is a simplified version of what can be found in https://github.com/DLR-RM/rl-baselines3-zoo.

You can run this example as follows:
    $ python sb3_simple.py

"""
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.2e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


ENV_ID = "Pogema-v0"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "verbose": 1,
    "env": ENV_ID
}

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for A2C hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values.
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model.
    model = A2C(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))


[I 2023-10-31 21:54:20,111] A new study created in memory with name: no-name-466b23a5-262d-4128-85aa-19e661157efa
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 64        |
|    ep_rew_mean        | 0         |
| time/                 |           |
|    fps                | 4994      |
|    iterations         | 100       |
|    time_elapsed       | 5         |
|    total_timesteps    | 25600     |
| train/                |           |
|    entropy_loss       | -1.97e-19 |
|    explained_variance | 0         |
|    learning_rate      | 0.138     |
|    n_updates          | 99        |
|    policy_loss        | -0        |
|    value_loss         | 0.465     |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 64        |
|    ep_rew_mean        | 0         |
| time/                 |           |
|    fps         

[I 2023-10-31 21:54:43,668] Trial 0 finished with value: 0.0 and parameters: {'gamma': 0.007766388324533128, 'max_grad_norm': 4.8474070671681675, 'gae_lambda': 0.00217321112824313, 'exponent_n_steps': 8, 'lr': 0.13786081068620215, 'ent_coef': 1.3948922996470193e-06, 'ortho_init': False, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 0 with value: 0.0.


Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 30       |
|    mean_reward        | 0        |
| time/                 |          |
|    total_timesteps    | 60000    |
| train/                |          |
|    entropy_loss       | -1.61    |
|    explained_variance | 0.0244   |
|    learning_rate      | 4.92e-05 |
|    n_updates          | 58       |
|    policy_loss        | 0.031    |
|    value_loss         | 0.0436   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49.2     |
|    ep_rew_mean        | 0.35     |
| time/                 |          |
|    fps                | 4837     |
|    iterations         | 100      |
|    time_elapsed       | 21       |
|    total_timesteps    | 102400   |
| 

[I 2023-10-31 21:55:08,647] Trial 1 finished with value: 0.0 and parameters: {'gamma': 0.011581858761745256, 'max_grad_norm': 0.3556731284678197, 'gae_lambda': 0.019533437342307433, 'exponent_n_steps': 10, 'lr': 4.9182808011402786e-05, 'ent_coef': 4.0973433150248817e-08, 'ortho_init': False, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 0 with value: 0.0.


...

Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50.9     |
|    ep_rew_mean        | 0.274    |
| time/                 |          |
|    fps                | 4314     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 3200     |
| train/                |          |
|    entropy_loss       | -1.58    |
|    explained_variance | -0.203   |
|    learning_rate      | 0.000259 |
|    n_updates          | 99       |
|    policy_loss        | 0.548    |
|    value_loss         | 0.231    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 47.8     |
|    ep_rew_mean        | 0.34     |
| time/                 |          |
|    fps                | 4323     |
| 

[I 2023-10-31 22:03:51,012] Trial 20 finished with value: 0.6666666666666666 and parameters: {'gamma': 0.03619853995916698, 'max_grad_norm': 0.8746145594736527, 'gae_lambda': 0.028187121356535644, 'exponent_n_steps': 5, 'lr': 0.00025939043596031973, 'ent_coef': 2.8784426309308087e-05, 'ortho_init': False, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 20 with value: 0.6666666666666666.


Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 47.6     |
|    ep_rew_mean        | 0.388    |
| time/                 |          |
|    fps                | 4107     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 3200     |
| train/                |          |
|    entropy_loss       | -1.6     |
|    explained_variance | -1.38    |
|    learning_rate      | 0.000189 |
|    n_updates          | 99       |
|    policy_loss        | -0.0785  |
|    value_loss         | 0.00574  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 48.1     |
|    ep_rew_mean        | 0.4      |
| time/                 |          |
|    fps                | 4181     |
| 

[I 2023-10-31 22:04:20,676] Trial 21 finished with value: 0.0 and parameters: {'gamma': 0.04369849934381889, 'max_grad_norm': 1.0320557281366587, 'gae_lambda': 0.027222553370946506, 'exponent_n_steps': 5, 'lr': 0.0001890419040917064, 'ent_coef': 3.0151527434143574e-05, 'ortho_init': False, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 20 with value: 0.6666666666666666.


Number of finished trials:  22
Best trial:
  Value:  0.6666666666666666
  Params: 
    gamma: 0.03619853995916698
    max_grad_norm: 0.8746145594736527
    gae_lambda: 0.028187121356535644
    exponent_n_steps: 5
    lr: 0.00025939043596031973
    ent_coef: 2.8784426309308087e-05
    ortho_init: False
    net_arch: tiny
    activation_fn: tanh
  User attrs:
    gamma_: 0.963801460040833
    gae_lambda_: 0.9718128786434643
    n_steps: 32


### Train agent with best hyper parameters

In [31]:
a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1,
    gamma=0.03619853995916698,
    max_grad_norm=0.8746145594736527,
    gae_lambda=0.028187121356535644,
    learning_rate=0.00025939043596031973,
    ent_coef=2.8784426309308087e-05,
    n_steps=32,
    #policy_kwargs={"net_arch": [{"pi": [64], "vf": [64]}], "activation_fn": nn.Tanh, "ortho_init": False}
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [32]:
# Train agent and save it
a2c_model.learn(int(1.2e5))
a2c_model.save("saved/a2c_baseline")

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 25.2     |
|    ep_rew_mean        | 0.3      |
| time/                 |          |
|    fps                | 781      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 3200     |
| train/                |          |
|    entropy_loss       | -1.6     |
|    explained_variance | -784     |
|    learning_rate      | 0.000259 |
|    n_updates          | 99       |
|    policy_loss        | 0.0966   |
|    value_loss         | 0.0192   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 25       |
|    ep_rew_mean        | 0.3      |
| time/                 |          |
|    fps                | 782      |
|    iterations         | 200      |
|    time_elapsed       | 8        |
|    total_timesteps    | 6400     |
| train/                |          |
|

#### Testing 

In [33]:
a2c_model = A2C.load("saved/a2c_baseline")

env.reset()

mean_reward, std_reward = evaluate_policy(a2c_model, env, deterministic=True, n_eval_episodes=20)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:0.20 +/- 0.40


In [34]:
# RANDOM SEED
from IPython.display import SVG, display
from pogema.animation import AnimationMonitor, AnimationConfig

env = AnimationMonitor(env)

def evaluate_success_rate(model, env, num_episodes=100):
    success_count = 0
    step_array = []
    for i in range(num_episodes):
        print(f'---{i}---')
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 100
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                env.save_animation(f"render{i}.svg", AnimationConfig(egocentric_idx=0))
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(a2c_model, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

---0---
2 100 0 False
1 99 0 False
2 98 0 False
1 97 0 False
4 96 0 False
3 95 0 False
2 94 0 False
3 93 0 False
3 92 0 False
3 91 0 False
3 90 0 True
---1---
1 100 1 False
2 99 1 False
1 98 1 False
1 97 1 False
1 96 1 False
2 95 1 False
1 94 1 False
3 93 1 False
1 92 1 False
2 91 1 False
2 90 1 False
4 89 1 False
2 88 1 False
1 87 1 False
1 86 1 False
1 85 1 False
3 84 1 False
4 83 1 False
3 82 1 False
1 81 1 False
1 80 1 False
2 79 1 False
4 78 1 False
1 77 1 False
3 76 1 False
3 75 1 False
1 74 1 False
2 73 1 False
2 72 1 False
2 71 1 False
1 70 1 False
1 69 1 False
1 68 1 False
2 67 1 False
2 66 1 False
2 65 1 False
2 64 1 False
1 63 1 False
1 62 1 False
2 61 1 False
1 60 1 False
2 59 1 False
3 58 1 False
2 57 1 False
3 56 1 False
4 55 1 False
1 54 1 False
1 53 1 False
1 52 1 False
3 51 1 False
1 50 1 False
2 49 1 False
2 48 1 False
3 47 1 False
1 46 1 False
1 45 1 False
2 44 1 False
1 43 1 False
2 42 1 False
1 41 1 False
2 40 1 False
2 39 1 False
2 38 1 False
4 37 1 False
0 36 1 F